In [40]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [41]:
import pandas as pd
import numpy as np

import commonfunc

In [42]:
data_set_path='C:/Users/shrus/Documents/Synthetic-data-generation/'

In [43]:
procedureevents_mv_df = commonfunc.read_csv_no_rowid(data_set_path+"sampled_data_csv_100/procedureevents_mv.csv")

In [53]:
procedureevents_mv_df

,starttime,endtime,itemid,value,valueuom,orderid,linkorderid,ordercategoryname,ordercategorydescription,isopenbag,statusdescription
0,2109-11-16 09:00:00,2109-11-16 21:17:00,224277,737.0,min,1242187,1242187,Peripheral Lines,Task,1,FinishedRunning
1,2109-11-16 09:16:00,2109-11-17 16:00:00,224277,1844.0,min,2598864,2598864,Peripheral Lines,Task,1,FinishedRunning
2,2109-11-16 20:00:00,2109-11-16 20:28:00,224277,28.0,min,4008354,4008354,Peripheral Lines,Task,1,FinishedRunning
3,2109-11-16 22:30:00,2109-11-16 22:31:00,225401,1.0,None,2525424,2525424,Procedures,Electrolytes,0,FinishedRunning
4,2109-11-16 22:30:00,2109-11-16 22:31:00,225454,1.0,None,1460562,1460562,Procedures,Electrolytes,0,FinishedRunning
...,...,...,...,...,...,...,...,...,...,...,...
586,2130-08-19 21:30:00,2130-08-20 07:17:00,224263,587.0,min,1331300,1331300,Invasive Lines,Task,1,FinishedRunning
587,2130-08-19 21:45:00,2130-08-20 01:32:00,225792,227.0,min,4867187,4867187,Ventilation,Task,1,FinishedRunning
588,2130-08-19 23:50:00,2130-08-20 07:17:00,224263,447.0,min,8544438,8544438,Invasive Lines,Task,1,FinishedRunning
589,2121-11-23 20:09:00,2121-11-24 00:38:00,224274,269.0,min,6899638,6899638,Peripheral Lines,Task,1,FinishedRunning


## Data Preprocess

In [45]:
# Drop some columns
procedureevents_mv_df.drop(['location', 'locationcategory', 'storetime', 'cgid', 'secondaryordercategoryname', 
    'continueinnextdept', 'cancelreason', 'comments_editedby', 'comments_canceledby', 'comments_date'], axis=1, inplace=True)

In [46]:
# Missing values
procedureevents_mv_df.isnull().sum()

subject_id                  0
hadm_id                     0
icustay_id                  2
starttime                   0
endtime                     0
itemid                      0
value                       0
valueuom                    0
orderid                     0
linkorderid                 0
ordercategoryname           0
ordercategorydescription    0
isopenbag                   0
statusdescription           0
dtype: int64

In [47]:
procedureevents_mv_df.dtypes

subject_id                    int64
hadm_id                       int64
icustay_id                  float64
starttime                    object
endtime                      object
itemid                        int64
value                       float64
valueuom                     object
orderid                       int64
linkorderid                   int64
ordercategoryname            object
ordercategorydescription     object
isopenbag                     int64
statusdescription            object
dtype: object

In [48]:
procedureevents_mv_df['starttime'] = pd.to_datetime(procedureevents_mv_df['starttime'])
procedureevents_mv_df['endtime'] = pd.to_datetime(procedureevents_mv_df['endtime'])

In [49]:
# process special data
invalid_index = procedureevents_mv_df.loc[(procedureevents_mv_df['starttime'] > procedureevents_mv_df['endtime']) == True].index
procedureevents_mv_df.drop(invalid_index, inplace=True, axis=0)

In [50]:
procedureevents_mv_df

,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,value,valueuom,orderid,linkorderid,ordercategoryname,ordercategorydescription,isopenbag,statusdescription
0,87197,176931,209341.0,2109-11-16 09:00:00,2109-11-16 21:17:00,224277,737.0,min,1242187,1242187,Peripheral Lines,Task,1,FinishedRunning
1,87197,176931,209341.0,2109-11-16 09:16:00,2109-11-17 16:00:00,224277,1844.0,min,2598864,2598864,Peripheral Lines,Task,1,FinishedRunning
2,87197,176931,209341.0,2109-11-16 20:00:00,2109-11-16 20:28:00,224277,28.0,min,4008354,4008354,Peripheral Lines,Task,1,FinishedRunning
3,87197,176931,209341.0,2109-11-16 22:30:00,2109-11-16 22:31:00,225401,1.0,None,2525424,2525424,Procedures,Electrolytes,0,FinishedRunning
4,87197,176931,209341.0,2109-11-16 22:30:00,2109-11-16 22:31:00,225454,1.0,None,1460562,1460562,Procedures,Electrolytes,0,FinishedRunning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,58667,197356,249803.0,2130-08-19 21:30:00,2130-08-20 07:17:00,224263,587.0,min,1331300,1331300,Invasive Lines,Task,1,FinishedRunning
587,58667,197356,249803.0,2130-08-19 21:45:00,2130-08-20 01:32:00,225792,227.0,min,4867187,4867187,Ventilation,Task,1,FinishedRunning
588,58667,197356,249803.0,2130-08-19 23:50:00,2130-08-20 07:17:00,224263,447.0,min,8544438,8544438,Invasive Lines,Task,1,FinishedRunning
589,52058,146270,292639.0,2121-11-23 20:09:00,2121-11-24 00:38:00,224274,269.0,min,6899638,6899638,Peripheral Lines,Task,1,FinishedRunning


In [51]:
procedureevents_mv_df = procedureevents_mv_df.drop(['subject_id', 'hadm_id', 'icustay_id'], axis=1)

In [52]:
procedureevents_mv_df.columns

Index(['starttime', 'endtime', 'itemid', 'value', 'valueuom', 'orderid',
       'linkorderid', 'ordercategoryname', 'ordercategorydescription',
       'isopenbag', 'statusdescription'],
      dtype='object')

## Metadata

In [16]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

In [17]:
metadata.detect_from_dataframe(data=procedureevents_mv_df)

In [19]:
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "starttime": {
            "sdtype": "datetime"
        },
        "endtime": {
            "sdtype": "datetime"
        },
        "itemid": {
            "sdtype": "numerical"
        },
        "value": {
            "sdtype": "numerical"
        },
        "valueuom": {
            "sdtype": "categorical"
        },
        "orderid": {
            "sdtype": "numerical"
        },
        "linkorderid": {
            "sdtype": "numerical"
        },
        "ordercategoryname": {
            "sdtype": "categorical"
        },
        "ordercategorydescription": {
            "sdtype": "categorical"
        },
        "isopenbag": {
            "sdtype": "numerical"
        },
        "statusdescription": {
            "sdtype": "categorical"
        }
    }
}

In [25]:
metadata.update_column(
    column_name='orderid',
    sdtype='categorical'
)

metadata.update_column(
    column_name='linkorderid',
    sdtype='categorical'
)

metadata.update_column(
    column_name='isopenbag',
    sdtype='categorical'
)
metadata.update_column(
    column_name='itemid',
    sdtype='categorical'
)
metadata.update_column(
    column_name='value',
    sdtype='categorical'
)

# Building model

## Set constraints

In [26]:

fixed_constraint1 = {
    'constraint_class': 'FixedCombinations',
    'constraint_parameters': {
        'column_names': ['itemid', 'value', 'valueuom']
    }
}

fixed_constraint2 = {
    'constraint_class': 'FixedCombinations',
    'constraint_parameters': {
        'column_names': ['orderid', 'linkorderid', 'ordercategoryname', 'ordercategorydescription']
    }
}

## Modelling

In [27]:
data=procedureevents_mv_df

In [29]:
from sdv.lite import SingleTablePreset

synthesizer1 = SingleTablePreset(metadata, name='FAST_ML')
synthesizer1.add_constraints(constraints=[
    fixed_constraint1, fixed_constraint2
])
synthesizer1.fit(data)


synthetic_data1 = synthesizer1.sample(num_rows=len(data))

Sampling rows: 100%|██████████| 591/591 [00:00<00:00, 13460.66it/s]


In [30]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer2 = GaussianCopulaSynthesizer(metadata)
synthesizer2.add_constraints(constraints=[
    fixed_constraint1, fixed_constraint2
])
synthesizer2.fit(data)

synthetic_data2 = synthesizer2.sample(num_rows=len(data))

Sampling rows: 100%|██████████| 591/591 [00:00<00:00, 12102.68it/s]


In [31]:
from sdv.single_table import CTGANSynthesizer

synthesizer3 = CTGANSynthesizer(metadata)
synthesizer3.add_constraints(constraints=[
    fixed_constraint1, fixed_constraint2
])
synthesizer3.fit(data)

synthetic_data3 = synthesizer3.sample(num_rows=len(data))

Sampling rows: 100%|██████████| 591/591 [00:01<00:00, 378.01it/s]


In [32]:
from sdv.single_table import TVAESynthesizer

synthesizer4 = TVAESynthesizer(metadata)
synthesizer4.add_constraints(constraints=[
    fixed_constraint1, fixed_constraint2
])
synthesizer4.fit(data)

synthetic_data4 = synthesizer4.sample(num_rows=len(data))

Sampling rows: 100%|██████████| 591/591 [00:00<00:00, 2951.93it/s]


In [33]:
from sdv.single_table import CopulaGANSynthesizer

synthesizer5 = CopulaGANSynthesizer(metadata)
synthesizer5.add_constraints(constraints=[
    fixed_constraint1, fixed_constraint2
])
synthesizer5.fit(data)

synthetic_data5 = synthesizer5.sample(num_rows=len(data))

Sampling rows: 100%|██████████| 591/591 [00:01<00:00, 393.87it/s]


# Evaluation of models

In [34]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=data,
    synthetic_data=synthetic_data1,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]



Overall Quality Score: 68.42%

Properties:
Column Shapes: 84.02%
Column Pair Trends: 52.83%


In [35]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=data,
    synthetic_data=synthetic_data2,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.67it/s]



Overall Quality Score: 64.93%

Properties:
Column Shapes: 79.22%
Column Pair Trends: 50.64%


In [36]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=data,
    synthetic_data=synthetic_data3,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.56it/s]



Overall Quality Score: 65.64%

Properties:
Column Shapes: 79.83%
Column Pair Trends: 51.45%


In [37]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=data,
    synthetic_data=synthetic_data4,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]



Overall Quality Score: 54.18%

Properties:
Column Shapes: 65.77%
Column Pair Trends: 42.58%


In [38]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=data,
    synthetic_data=synthetic_data5,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.56it/s]



Overall Quality Score: 65.27%

Properties:
Column Shapes: 79.56%
Column Pair Trends: 50.98%


In [39]:
synthesizer1.save('C:/Users/shrus/Documents/Synthetic-data-generation/models/procedureevents_mv/'+'procedureevents_mv_best_tabularpreset.pkl')
